<a href="https://colab.research.google.com/github/MAmmarx/Facial-Emotions-Recognition/blob/main/Facial_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
dataset_dir = '/content/drive/MyDrive/Dataset'
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')
print(train_dir)
print(test_dir)

/content/drive/MyDrive/Dataset/train
/content/drive/MyDrive/Dataset/test


In [7]:
IMG_HEIGHT = 100
IMG_WIDTH = 100
BATCH_SIZE = 100
EPOCHS = 15

In [8]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    seed=123
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    seed=123
)

Found 1799 files belonging to 7 classes.
Found 488 files belonging to 7 classes.


In [9]:
class_names = train_dataset.class_names
print(f"Found classes: {class_names}")

Found classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [10]:
model = tf.keras.models.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(7, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,293,511 (4.93 MB)

 Trainable params: 1,293,511 (4.93 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset
)

Starting training...
Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 363s 20s/step - accuracy: 0.1606 - loss: 1.9527 - val_accuracy: 0.1680 - val_loss: 1.9708
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 291ms/step - accuracy: 0.1697 - loss: 1.9438 - val_accuracy: 0.1926 - val_loss: 1.9625
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 270ms/step - accuracy: 0.1856 - loss: 1.9283 - val_accuracy: 0.1332 - val_loss: 1.9576
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 299ms/step - accuracy: 0.2030 - loss: 1.9165 - val_accuracy: 0.1639 - val_loss: 1.9440
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 288ms/step - accuracy: 0.2278 - loss: 1.8744 - val_accuracy: 0.2049 - val_loss: 1.9112
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 256ms/step - accuracy: 0.2617 - loss: 1.8280 - val_accuracy: 0.2541 - val_loss: 1.8477
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 293ms/step - accuracy: 0.2738 - loss: 1.7855 - val_accuracy: 0.2295 - val_loss: 1.8555
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 275ms/step - accuracy: 0.2912 - loss: 

In [14]:
model.save('Facial_Emotion_Recognition_CNN_Modle.h5')
print("Model saved as 'Facial_Emotion_Recognition_CNN_Modle'.h5'")

test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

Model saved as 'Facial_Emotion_Recognition_CNN_Modle'.h5'
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step - accuracy: 0.3383 - loss: 1.7583
Test Accuracy: 0.3545


In [15]:
import tensorflow as tf

model = tf.keras.models.load_model('Facial_Emotion_Recognition_CNN_Modle.h5')